In [ ]:
import h5py    
import numpy as np 
import matplotlib.pyplot as plt
import nbimporter
import gc

from keras.layers import Input, Dense, Dropout, Average, LeakyReLU
from keras.models import Model, Sequential, load_model
from keras.initializers import TruncatedNormal
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K 

from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterSciNotation as LogFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import tensorflow as tf
from sklearn.metrics import roc_curve, auc, roc_auc_score
from check_efficiency_after_epoch_end import IntervalEvaluation

In [ ]:
# Prepare GPU environment and define amount of memory to use
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # specify which GPU(s) to be used
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.42)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
# Load input features from file
features_filename = 'final_features_cwola_and_ae/all_input_features.hdf5'
h5_results = h5py.File(features_filename, 'r')

# Store features info
signal = h5_results['signal'][:]
h5_results.close()

# Load extra background
filename = "final_features_cwola_and_ae/all_input_features_extra_QCD.hdf5"
h5_results = h5py.File(filename, 'r')

# Store reconstruction info
background = h5_results['background'][:]

h5_results.close()

In [ ]:
# Print shape of data arrays
print('Signal features shape: %s' % (signal.shape,))
print('Background features shape: %s' % (background.shape,))

In [ ]:
# Check the amount of memory used by the arrays
print('Memory used by signal array: %.2f MB' % (signal.nbytes / (10**3 * 1024)))
print('Memory used by background array: %.2f MB' % (background.nbytes / (10**3 * 1024)))

In [ ]:
# Print sample signal and background event to make sure everything is fine
print('Signal sample event:\n %s' % signal[-1])
print('Background sample event:\n %s' % background[-1])

In [ ]:
# Study each feature of jet 1 before preprocessing data
print('BACKGROUND:')
print('mj: (min, max) = (%f, %f) | Only %i below 1' % (background[:,0].min(), background[:,0].max(), background[:,0][background[:,0] < 1].shape[0]))
print('t2: (min, max) = (%f, %f)   | Only %i above 15' % (background[:,1].min(), background[:,1].max(), background[:,1][background[:,1] > 15].shape[0]))
print('tau21: (min, max) = (%f, %f)' % (background[:,2].min(), background[:,2].max()))
print('tau32: (min, max) = (%f, %f)' % (background[:,3].min(), background[:,3].max()))
print('tau43: (min, max) = (%f, %f)' % (background[:,4].min(), background[:,4].max()))
print('ntrk: (min, max) = (%i, %i)' % (background[:,5].min(), background[:,5].max()))

# Study each feature of jet 1 before preprocessing data
print('SIGNAL:')
print('mj: (min, max) = (%f, %f) | Only %i below 1' % (signal[:,0].min(), signal[:,0].max(), signal[:,0][signal[:,0] < 1].shape[0]))
print('t2: (min, max) = (%f, %f)   | Only %i above 15' % (signal[:,1].min(), signal[:,1].max(), signal[:,1][signal[:,1] > 15].shape[0]))
print('tau21: (min, max) = (%f, %f)' % (signal[:,2].min(), signal[:,2].max()))
print('tau32: (min, max) = (%f, %f)' % (signal[:,3].min(), signal[:,3].max()))
print('tau43: (min, max) = (%f, %f)' % (signal[:,4].min(), signal[:,4].max()))
print('ntrk: (min, max) = (%i, %i)' % (signal[:,5].min(), signal[:,5].max()))

In [ ]:
# Study each feature of jet 2 before preprocessing data
print('BACKGROUND:')
print('mj: (min, max) = (%f, %f) | Only %i below 0' % (background[:,9].min(), background[:,9].max(), background[:,9][background[:,9] < 0].shape[0]))
print('t2: (min, max) = (%f, %f)' % (background[:,10].min(), background[:,10].max()))
print('tau21: (min, max) = (%f, %f)' % (background[:,11].min(), background[:,11].max()))
print('tau32: (min, max) = (%f, %f)' % (background[:,12].min(), background[:,12].max()))
print('tau43: (min, max) = (%f, %f)' % (background[:,13].min(), background[:,13].max()))
print('ntrk: (min, max) = (%i, %i)' % (background[:,14].min(), background[:,14].max()))

# Study each feature of jet 2 before preprocessing data
print('SIGNAL:')
print('mj: (min, max) = (%f, %f) | Only %i below 0' % (signal[:,9].min(), signal[:,9].max(), signal[:,9][signal[:,9] < 0].shape[0]))
print('t2: (min, max) = (%f, %f)' % (signal[:,10].min(), signal[:,10].max()))
print('tau21: (min, max) = (%f, %f)' % (signal[:,11].min(), signal[:,11].max()))
print('tau32: (min, max) = (%f, %f)' % (signal[:,12].min(), signal[:,12].max()))
print('tau43: (min, max) = (%f, %f)' % (signal[:,13].min(), signal[:,13].max()))
print('ntrk: (min, max) = (%i, %i)' % (signal[:,14].min(), signal[:,14].max()))

In [ ]:
#Let's make some very simple plots.
fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = np.concatenate((background[:,0], background[:,9]),axis=0)
all_sig = np.concatenate((signal[:,0], signal[:,9]),axis=0)
plt.hist(all_back, range = (0, 1000), 
         bins=50, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1000), 
         bins=50, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$m_{J}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.xlim(xmax=1000)

ax = fig.add_subplot(1, 2, 2)
all_back = np.concatenate((background[:,1], background[:,10]),axis=0)
all_sig = np.concatenate((signal[:,1], signal[:,10]),axis=0)
plt.hist(all_back, range = (0, 8), 
         bins=100, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 8), 
         bins=100, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\sqrt{\tau_{1}^{(2)}} / \tau_{1}^{(1)}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.xlim(xmin=-0.05, xmax=8)

plt.show()


fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = np.concatenate((background[:,2], background[:,11]),axis=0)
all_sig = np.concatenate((signal[:,2], signal[:,11]),axis=0)
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{21}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)

ax = fig.add_subplot(1, 2, 2)
all_back = np.concatenate((background[:,3], background[:,12]),axis=0)
all_sig = np.concatenate((signal[:,3], signal[:,12]),axis=0)
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{32}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)

plt.show()


fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = np.concatenate((background[:,4], background[:,13]),axis=0)
all_sig = np.concatenate((signal[:,4], signal[:,13]),axis=0)
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{43}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmax=1)

ax = fig.add_subplot(1, 2, 2)
all_back = np.concatenate((background[:,5], background[:,14]),axis=0)
all_sig = np.concatenate((signal[:,5], signal[:,14]),axis=0)
plt.hist(all_back, range = (0, 200), 
         bins=50, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 200), 
         bins=50, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$n_{trk}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')

plt.show()


In [ ]:
# Check background distribution on mj1 and mj2, since we might be able to get rid of some "outliers" or weird back.
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15.1, 6))
plt.subplots_adjust(wspace = 0.3)

# Prepare inputs
mj1_background = background[:,0]
mj2_background = background[:,9]

h1 = ax1.hist2d(mj1_background, mj2_background, bins=(100, 100), range=[[10, 1000], [10, 600]], norm=LogNorm(), cmap=plt.cm.jet, label='background')
divider = make_axes_locatable(ax1)  # create an axes on the right side of ax.
cax = divider.append_axes("right", size="8%", pad=0.1)
cb = plt.colorbar(h1[3], ax=ax1, cax = cax, format=LogFormatter(10))
ax1.set_xlabel(r'$m_{J1}$ [GeV]') 
ax1.set_ylabel(r'$m_{J2}$ [GeV]')
ax1.set_xlim(xmin=10, xmax=1000)
ax1.set_ylim(ymin=10, ymax=600)


mj1_signal = signal[:,0]
mj2_signal = signal[:,9]

h2 = ax2.hist2d(mj1_signal, mj2_signal, bins=(100, 100), norm=LogNorm(), cmap=plt.cm.jet, label='signal')
divider = make_axes_locatable(ax2)  # create an axes on the right side of ax.
cax = divider.append_axes("right", size="8%", pad=0.1)
cb = plt.colorbar(h1[3], ax=ax2, cax = cax, format=LogFormatter(10))
ax2.set_xlabel(r'$m_{J1}$ [GeV]') 
ax2.set_ylabel(r'$m_{J2}$ [GeV]')
ax2.set_xlim(xmin=10, xmax=1000)
ax2.set_ylim(ymin=10, ymax=600)

plt.show()

In [ ]:
# Take only a few input features
back_1 = background[:,0].reshape((1,len(background)))    # mj (1)
back_2 = background[:,1].reshape((1,len(background)))    # t2 (1)
back_3 = background[:,2].reshape((1,len(background)))    # tau21 (1)
back_4 = background[:,3].reshape((1,len(background)))    # tau32 (1)
back_5 = background[:,4].reshape((1,len(background)))    # tau43 (1)
back_6 = background[:,5].reshape((1,len(background)))    # ntrk (1)

back_7 = background[:,9].reshape((1,len(background)))    # mj (2)
back_8 = background[:,10].reshape((1,len(background)))   # t2 (2)
back_9 = background[:,11].reshape((1,len(background)))   # tau21 (2)
back_10 = background[:,12].reshape((1,len(background)))  # tau32 (2)
back_11 = background[:,13].reshape((1,len(background)))  # tau43 (2)
back_12 = background[:,14].reshape((1,len(background)))  # ntrk (2)

back_mjj = background[:,18].reshape((1,len(background)))

sig_1 = signal[:,0].reshape((1,len(signal)))    # mj (1)
sig_2 = signal[:,1].reshape((1,len(signal)))    # t2 (1)
sig_3 = signal[:,2].reshape((1,len(signal)))    # tau21 (1)
sig_4 = signal[:,3].reshape((1,len(signal)))    # tau32 (1)
sig_5 = signal[:,4].reshape((1,len(signal)))    # tau43 (1)
sig_6 = signal[:,5].reshape((1,len(signal)))    # ntrk (1)

sig_7 = signal[:,9].reshape((1,len(signal)))    # mj (2)
sig_8 = signal[:,10].reshape((1,len(signal)))   # t2 (2)
sig_9 = signal[:,11].reshape((1,len(signal)))   # tau21 (2)
sig_10 = signal[:,12].reshape((1,len(signal)))  # tau32 (2)
sig_11 = signal[:,13].reshape((1,len(signal)))  # tau43 (2)
sig_12 = signal[:,14].reshape((1,len(signal)))  # ntrk (2)

sig_mjj = signal[:,18].reshape((1,len(signal)))

# Input features: [mj1, t2(1), tau21(1), tau32(1), tau43(1), ntrk1] 
#                 [mj2, t2(2), tau21(2), tau32(2), tau43(2), ntrk2, mjj]

background = np.concatenate((back_1, back_2, back_3, back_4, back_5, back_6, back_7, back_8, back_9, back_10, back_11, back_12, back_mjj), axis=0).T
signal = np.concatenate((sig_1, sig_2, sig_3, sig_4, sig_5, sig_6, sig_7, sig_8, sig_9, sig_10, sig_11, sig_12, sig_mjj), axis=0).T

In [ ]:
# Print sample signal and background event to make sure everything is fine
print('Background features shape before mj cut: %s' % (background.shape,))
print('Signal features shape before mj cut: %s' % (signal.shape,))

In [ ]:
# Input features: [mj1, t2(1), tau21(1), tau32(1), tau43(1), ntrk1] 
#                 [mj2, t2(2), tau21(2), tau32(2), tau43(2), ntrk2, mjj]

##################################### mj #####################################
# Plot the reduced set of input features that we use to fee the AE
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(1, 2, 1)
all_back = background[:,0]
all_sig = signal[:,0]
plt.hist(all_back, range = (0, 1600), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1600), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$m_{J1}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.semilogy()
plt.xlim(xmax=1600)


ax = fig.add_subplot(1, 2, 2)
all_back = background[:,6]
all_sig = signal[:,6]
plt.hist(all_back, range = (0, 1600), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1600), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$m_{J2}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.semilogy()
plt.xlim(xmax=1600)

plt.show()


##################################### t2 #####################################
fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = background[:,1]
all_sig = signal[:,1]
plt.hist(all_back, range = (0, 8), 
         bins=100, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 8), 
         bins=100, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\sqrt{\tau_{1}^{(2)}} / \tau_{1}^{(1)}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.xlim(xmin=-0.05, xmax=8)

ax = fig.add_subplot(1, 2, 2)
all_back = background[:,7]
all_sig = signal[:,7]
plt.hist(all_back, range = (0, 8), 
         bins=100, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 8), 
         bins=100, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\sqrt{\tau_{1}^{(2)}} / \tau_{1}^{(1)}$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')
plt.xlim(xmin=-0.05, xmax=8)

plt.show()


##################################### tau21 #####################################

fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = background[:,2]
all_sig = signal[:,2]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{21} (1)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)


ax = fig.add_subplot(1, 2, 2)
all_back = background[:,8]
all_sig = signal[:,8]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{21} (2)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)

plt.show()


##################################### tau32 #####################################

fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = background[:,3]
all_sig = signal[:,3]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{32} (1)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)


ax = fig.add_subplot(1, 2, 2)
all_back = background[:,9]
all_sig = signal[:,9]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{32} (2)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)

plt.show()



##################################### tau43 #####################################

fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = background[:,4]
all_sig = signal[:,4]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{43} (1)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)


ax = fig.add_subplot(1, 2, 2)
all_back = background[:,10]
all_sig = signal[:,10]
plt.hist(all_back, range = (0, 1), 
         bins=60, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 1), 
         bins=60, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$\tau_{43} (2)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper left')
plt.xlim(xmin=-0.05, xmax=1)

plt.show()


##################################### ntrk #####################################

fig = plt.figure(figsize=(12,5))

ax = fig.add_subplot(1, 2, 1)
all_back = background[:,5]
all_sig = signal[:,5]
plt.hist(all_back, range = (0, 200), 
         bins=50, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 200), 
         bins=50, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$n_{trk}(1)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')

ax = fig.add_subplot(1, 2, 2)
all_back = background[:,11]
all_sig = signal[:,11]
plt.hist(all_back, range = (0, 200), 
         bins=50, facecolor='r', alpha=0.2,label='background')
plt.hist(all_sig, range = (0, 200), 
         bins=50, facecolor='b', alpha=0.2,label='signal')
plt.xlabel(r'$n_{trk}(2)$')
plt.ylabel('Relative ocurrence')
plt.legend(loc='upper right')

plt.show()

In [ ]:
# Define function to shuffle array elements in unison
def shuffle_2D(a, b):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces]

def shuffle_3D(a, b, c):
    n_elem = a.shape[0]
    indeces = np.random.choice(n_elem, size=n_elem, replace=False)
    return a[indeces], b[indeces], c[indeces]

In [ ]:
###################################################################################################################
###################################################################################################################
#################################################                 #################################################
#################################################    RUN CWOLA    #################################################
#################################################                 #################################################
###################################################################################################################
###################################################################################################################

In [ ]:
# Define number of signal and background events for the S/B scan. S/B benchmarks denotes as B1-B7
back_scan = [537304, 537304, 537304, 537304, 537304, 537304, 537304]
sig_scan = [730, 580, 440, 350, 265, 175, 1]

In [ ]:
# Define run number (run this same notebook in "parallel", files will be saved with suffix "_run")
run = 1

# Create folders to store the trained models
for i in range(1, 1+len(back_scan)):
    !mkdir CWoLa_signal_m500_models/models_B{i}_%d % run              # create folder to store trained models
    !mkdir CWoLa_signal_m500_models/ensemble_models_B{i}_%d % run     # create folder to store ensemble models
    !mkdir CWoLa_signal_m500_models/extra_info_B{i}_%d % run          # create folder to store file with some info

In [ ]:
# Remove signal events outside the range 2800 < mjj < 5200. Note: the initial sample of background events has events
# in the range 1000 < mJJ < 8000, and peaks at ~2700 GeV. The signal distribution is centered around ~ 3500 GeV. As
# a consequence, we will only consider events in the relevant mJJ range: 2800 < mJJ < 5200 works well for us.
signal = signal[(signal[:,12] > mass_min) & (signal[:,12] < mass_max)]
background = background[(background[:,12] > mass_min) & (background[:,12] < mass_max)]

# Make copies of these arrays to take random subsamples of events later on
signal_ref = np.copy(signal)
background_ref = np.copy(background)

In [ ]:
######################################################################################################
############################################   TRAINING   ############################################
######################################################################################################

# Define training variables
kfolds = 5              # number of kfolds for the nested cross-validations procedure
ntries = 10             # number of models to train in each cross-validation round
nepochs = 700           # maximum number of epochs each model is trained for
nbatch = int(256*80)    # batch size
patience = 300          # wait n = patience epochs to see if validation performance improves before ending training
eff_rate = 0.01         # custom validation metric: signal region efficiency

# Define other variables
nbin = 30               # number of bins that we consider in the 2800 < mJJ < 5200
counter = 1             # this variable is only defined to keep track of the S/B benchmark and print some info


# Loop over S/B benchmarks
for evs in range(len(back_scan)):
    
    print('******************************  Training Benchmark %d  ******************************' % counter)
    
    # Shuffle input arrays
    signal = np.random.permutation(signal_ref)
    background = np.random.permutation(background_ref)
    
    # Define number of events for this benchmark
    n_signal = sig_scan[evs]                       # number of signal events used for training
    n_background = back_scan[evs]                  # number of background events used for training
    n_signal_extra = len(signal_ref) - n_signal    # the rest of signal events are used to compute the AUC metric

    # Define signal and background events
    signal_extra = signal[n_signal:(n_signal+n_signal_extra)]
    signal = signal[:n_signal]
    background = background[:n_background]
    
    print('Background features shape: %s' % (background.shape,))
    print('Signal features shape: %s' % (signal.shape,))
    print()
    
    
    #######################################################################################################
    ##########################################   Prepare input   ##########################################
    #######################################################################################################
    
    # For CWoLa, we will need k-folds. In this part of the code, we define an array with all the signal and 
    # background events that are used for training. We define the signal region (SR) and the sideband region (SB),
    # bin all data and split it in kfolds.

    # Define signal and background labels
    signal_labels = np.ones(len(signal))
    background_labels = np.zeros(len(background))
    
    # Mix signal and background inputs, add truth label as an extra column at the end
    X = np.concatenate((signal, background), axis=0)
    truth_labels = np.concatenate((signal_labels, background_labels), axis=0)
    truth_labels = truth_labels.reshape(truth_labels.shape[0], 1)
    X = np.concatenate((X, truth_labels.reshape(truth_labels.shape[0], 1)), axis=1)

    # Define extra signal labels
    signal_extra_labels = np.ones(len(signal_extra))
    truth_labels = signal_extra_labels.reshape(signal_extra_labels.shape[0], 1)
    signal_extra = np.concatenate((signal_extra, truth_labels), axis=1)

    # Find signal peak. If signal sample is large, calculate mean, otherwise use mean value from the full sample
    if (len(signal) > 100):
        mpeak = np.mean(signal[:,12], axis=0)
    else:
        mpeak = 3507.1692
    
    # Divide data in 30 bins of log(mjj) uniformly
    bins = np.logspace(np.log10(mass_min), np.log10(mass_max), num=nbin+1)
    bins_centers = np.array([0.5*(bins[i] + bins[i+1]) for i in range(0,len(bins)-1)])
    SR_lower_edge = 9     # index of "bins" array where the SR lower edge is stored
    SR_upper_edge = 13    # index of "bins" array where the SR upper edge is stored
    SB_lower_edge = 6     # index of "bins" array where the low SB lower edge is stored
    SB_upper_edge = 16    # index of "bins" array where the high SB upper edge is stored

    # Prepare data for cross-validation
    bin_list_kfolds = []  # bin_list_kfolds[bin number][number of kfolds, kfold size, number of features]

    for i in range(nbin):
        
        # Take all the events in a given bin
        w = X[(X[:,12] >= bins[i]) & (X[:,12] < bins[i+1])]
        w = np.random.permutation(w)
        
        # Split the list in kfolds of equal length
        kfold_size = int(len(w)/5)
        bin_list_kfolds.append(np.zeros((5, kfold_size, 14)))
        
        # Split all the events in a given bin in kfolds
        for j in range(kfolds):
            bin_list_kfolds[i][j] = w[j*kfold_size:(j+1)*kfold_size]
            
        pass
    
    
    #######################################################################################################
    ##########################################   Calculate S/B   ##########################################
    #######################################################################################################       
    
    # Compute S/B in SR
    dim_back = 0   # count total number of background events in the SR
    dim_sig = 0    # count total number of signal events in the SR

    for i in range(SR_lower_edge, SR_upper_edge):
        for k in range (kfolds):
            for j in range(bin_list_kfolds[i][k].shape[0]):
    
                if (bin_list_kfolds[i][k][j][13] == 0):
                    dim_back += 1
                else:
                    dim_sig += 1
    
    # Calculate useful information
    S_B = dim_sig/dim_back                  # S/B in the SR
    S_sqrt_B = dim_sig/np.sqrt(dim_back)    # S/sqrt(B) in the SR
    
    print('**********  Information  **********')
    print('Background events in SR: %d' % dim_back)
    print('Signal events in SR: %d' % dim_sig)
    print('S/sqrt(B) in SR: %.3f' % S_sqrt_B)
    print('S/B in SR: %.5f' % S_B)
    print('Signal events peak at m_JJ = %.0f GeV' % mpeak)
    print()
    
    
    #######################################################################################################
    ############################################   Training   #############################################
    #######################################################################################################
    
    # Define lists to store useful information
    best_val = [[[] for l in range(kfolds-1)] for k in range(kfolds)]        # store signal efficiency on val data
    models = [[[] for l in range(kfolds-1)] for k in range(kfolds)]          # store trained models
    models_history = [[[] for l in range(kfolds-1)] for k in range(kfolds)]  # store trained models history
    X_test_list = []                                                         # store test data fold in this list
    X_test_raw_list = []                                                     # store test raw data fold in this list

    # Implement cross-validation procedure
    for k in range(kfolds):                   # Loop over test fold
        for l in range(kfolds):               # Loop over validation fold
            
            #######################################################
            #####  Define training, validation and test sets  #####
            #######################################################
            
            # If both folds are "the same", go next (i.e. training and validation fold must be different)
            if (l == k):
                continue
        
            # Define lists to store events used for training, validation and testing in a given cross-validation 
            # round. Each list will have the following structure: 
            # list[bin number][kfold number, kfold size, number of features]
            train = []
            val = []
            test = []
            
            # Split data in each bin in n = kfolds parts, and add each part to one of the previously defined lists
            for i in range(nbin):
    
                # Initialize array in each list
                kfold_size = bin_list_kfolds[i].shape[1]
            
                train.append(np.zeros((3, kfold_size, 14)))
                val.append(np.zeros((1, kfold_size, 14)))
                test.append(np.zeros((1, kfold_size, 14)))
            
                # Select training data
                train_folds = [j for j in range(kfolds)]   # define index of the training folds
                train_folds.remove(k)                      # remove index of the testing fold
                train_folds.remove(l)                      # remove index of the validation fold
            
                # Fill arrays with data
                train[i] = np.array([bin_list_kfolds[i][j] for j in train_folds]).reshape((train[i].shape[0]*train[i].shape[1], 14))
                val[i] = bin_list_kfolds[i][l].reshape((val[i].shape[0]*val[i].shape[1], 14))
                test[i] = bin_list_kfolds[i][k].reshape((test[i].shape[0]*test[i].shape[1], 14))

                
            ################################################################
            #####  Define SR and SB using the previously defined sets  #####
            ################################################################

            # Define SR and short SB (lower and upper sidebands are defined separately, then joined together)
            SR_train = np.concatenate((train[SR_lower_edge], train[SR_lower_edge+1], train[SR_lower_edge+2], train[SR_lower_edge+3]), axis=0)
            SB_low_train = np.concatenate((train[SB_lower_edge], train[SB_lower_edge+1], train[SB_lower_edge+2]), axis=0)
            SB_up_train = np.concatenate((train[SR_upper_edge], train[SR_upper_edge+1], train[SR_upper_edge+2]), axis=0)
            SB_train = np.concatenate((SB_low_train, SB_up_train), axis=0)
        
            SR_val = np.concatenate((val[SR_lower_edge], val[SR_lower_edge+1], val[SR_lower_edge+2], val[SR_lower_edge+3]), axis=0)
            SB_low_val = np.concatenate((val[SB_lower_edge], val[SB_lower_edge+1], val[SB_lower_edge+2]), axis=0)
            SB_up_val = np.concatenate((val[SR_upper_edge], val[SR_upper_edge+1], val[SR_upper_edge+2]), axis=0)
            SB_val = np.concatenate((SB_low_val, SB_up_val), axis=0)
        
            # We train on SR + short SB and test on all bins
            SR_test = np.concatenate((test[SR_lower_edge], test[SR_lower_edge+1], test[SR_lower_edge+2], test[SR_lower_edge+3]), axis=0)
            SB_low_test = test[0]
            SB_up_test = test[SR_upper_edge]
        
            for m in range(1, SR_lower_edge):
                SB_low_test = np.concatenate((SB_low_test, test[m]), axis=0)
                
            for m in range(SR_upper_edge+1, nbin): 
                SB_up_test = np.concatenate((SB_up_test, test[m]), axis=0)
            
            SB_test = np.concatenate((SB_low_test, SB_up_test), axis=0)

            # Define number of events in SR and SB for the training, validation and test sets
            N_SR_train, N_low_train, N_up_train = len(SR_train), len(SB_low_train), len(SB_up_train)
            N_SR_val, N_low_val, N_up_val = len(SR_val), len(SB_low_val), len(SB_up_val)
            N_SR_test, N_low_test, N_up_test = len(SR_test), len(SB_low_test), len(SB_up_test)

            # Define total number of events used for training, validation and testing
            N_train = len(SR_train) + len(SB_low_train) + len(SB_up_train)
            N_val = len(SR_val) + len(SB_low_val) + len(SB_up_val)
            N_test = len(SR_test) + len(SB_low_test) + len(SB_up_test)
        
            # Calculate weights
            w_SR_train = (2*N_train) / (4*N_SR_train)
            w_low_train = N_train / (4*N_low_train)
            w_up_train = N_train / (4*N_up_train)
        
            w_SR_val = (2*N_val) / (4*N_SR_val)
            w_low_val = N_val / (4*N_low_val)
            w_up_val = N_val / (4*N_up_val)
        
            w_SR_test = (2*N_test) / (4*N_SR_test)
            w_low_test = N_test / (4*N_low_test)
            w_up_test = N_test / (4*N_up_test)
        
            # Define arrays with the weights
            SR_train_weights = np.ones(len(SR_train)) * w_SR_train
            SB_low_train_weights = np.ones(len(SB_low_train)) * w_low_train
            SB_up_train_weights = np.ones(len(SB_up_train)) * w_up_train
            SB_train_weights = np.concatenate((SB_low_train_weights, SB_up_train_weights))
        
            SR_val_weights = np.ones(len(SR_val)) * w_SR_val
            SB_low_val_weights = np.ones(len(SB_low_val)) * w_low_val
            SB_up_val_weights = np.ones(len(SB_up_val)) * w_up_val
            SB_val_weights = np.concatenate((SB_low_val_weights, SB_up_val_weights))
        
            # Define SR and SB labels (events in the SR labeled as 1, events in the SB labeled as 0)
            SR_train_labels = np.ones(len(SR_train))
            SB_train_labels = np.zeros(len(SB_train))
        
            SR_val_labels = np.ones(len(SR_val))
            SB_val_labels = np.zeros(len(SB_val))
            
            
            ###################################################################
            #####  Build the arrays that we will use to train the models  #####
            ################################################################### 

            # Build input arrays
            X_train = np.concatenate((SR_train, SB_train), axis=0)
            Y_train = np.concatenate((SR_train_labels, SB_train_labels))
            Z_train = np.concatenate((SR_train_weights, SB_train_weights))
        
            X_val = np.concatenate((SR_val, SB_val), axis=0)
            Y_val = np.concatenate((SR_val_labels, SB_val_labels))
            Z_val = np.concatenate((SR_val_weights, SB_val_weights))
        
            X_test = np.concatenate((SR_test, SB_test), axis=0)

            # Shuffle input arrays
            X_train, Y_train, Z_train = shuffle_3D(X_train, Y_train, Z_train)
            X_val, Y_val, Z_val = shuffle_3D(X_val, Y_val, Z_val)
            X_test = np.random.permutation(X_test)
        
    
            ############################################################
            #####  Preprocess the input features: standardization  #####
            ############################################################
    
            # Keep arrays with raw features
            X_train_raw = np.copy(X_train)
            X_val_raw = np.copy(X_val)
            X_test_raw = np.copy(X_test)
        
            # Calculate mean and std for each feature
            X_test_pre_mean = np.mean(np.concatenate((X_train_raw[:,:12], X_val_raw[:,:12]), axis=0), axis=0)
            X_test_pre_std = np.std(np.concatenate((X_train_raw[:,:12], X_val_raw[:,:12]), axis=0), axis=0)

            # Standardize input features (i.e. substract mean, divide by standard deviation)
            X_train[:,:12] = (X_train[:,:12] - np.mean(X_train_raw[:,:12], axis=0))/np.std(X_train_raw[:,:12], axis=0)
            X_val[:,:12] = (X_val[:,:12] - np.mean(X_train_raw[:,:12], axis=0))/np.std(X_train_raw[:,:12], axis=0) 
            X_test[:,:12] = (X_test[:,:12] - X_test_pre_mean) / X_test_pre_std
        
        
            #########################################################
            #####  Training phase: define NN and train models   #####
            #########################################################
            
            # Reset l index to fill arrays
            if (l > k):
                l = l-1
    
            # Train n = ntries models on same data and save the one with the best performance on the validation set
            for i in range(ntries):
            
                K.clear_session()
            
                # Build custom optimizer | Default: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
                myoptimizer = Adam(lr=0.001, beta_1=0.8, beta_2=0.99, epsilon=1e-08, decay=0.0005)
            
                # Build NN structure
                model = Sequential()
                model.add(Dense(64, input_dim=12, activation='relu', use_bias=True,
                                bias_initializer = TruncatedNormal(mean=0., stddev=0.04)))
                model.add(LeakyReLU(alpha=0.1))
                model.add(Dropout(0.2))
                model.add(Dense(32, activation='elu', use_bias=True,
                                bias_initializer = TruncatedNormal(mean=0., stddev=0.01)))
                model.add(Dropout(0.2))
                model.add(Dense(16, activation='elu', use_bias=True,
                                bias_initializer = TruncatedNormal(mean=0., stddev=0.01)))
                model.add(Dropout(0.2))
                model.add(Dense(4, activation='elu', use_bias=True,
                                bias_initializer = TruncatedNormal(mean=0., stddev=0.01)))
                model.add(Dense(1, activation='sigmoid'))
    
                # Define filename to save the trained model
                if (l < k):
                    checkpoint_filename = "CWoLa_signal_m500_models/models_B%d_%d/model_[" % (counter, run) + str(k) + "," + str(l) + "]_" + str(i) + ".h5"
                else:
                    checkpoint_filename = "CWoLa_signal_m500_models/models_B%d_%d/model_[" % (counter, run) + str(k) + "," + str(l+1) + "]_" + str(i) + ".h5" 

                # Monitor the evolution of our custom metric during training (this piece of code was written by Jack)
                ival = IntervalEvaluation(training_data=(X_train[:,:12], Y_train), 
                                          validation_data=(X_val[:,:12], Y_val),
                                          verbose=0, filename=checkpoint_filename,
                                          eff_rate=eff_rate, patience=patience,
                                          min_epoch=10, plot_period=2)
                
                # Compile and fit model
                model.compile(loss='binary_crossentropy', optimizer=myoptimizer)

                model_history = model.fit(X_train[:,:12], Y_train, epochs=nepochs, batch_size=nbatch,
                                          verbose=0, callbacks=[ival],
                                          sample_weight=Z_train, validation_data=(X_val[:,:12], Y_val, Z_val))
                
                # Save model history
                models_history[k][l].append(model_history)
                
                # Reload the trained model
                del model
                model = load_model(checkpoint_filename)
    
                # Calculate scores for training and validation data
                scores_train = model.predict(X_train[:,:12], batch_size=nbatch).flatten()
                scores_val = model.predict(X_val[:,:12], batch_size=nbatch).flatten()
                
                # Get score threshold above which only x% of the events survive in the SB (using validation data!)
                scores_sorted = np.sort(scores_val[Y_val == 0])[::-1]   # inverse sort, i.e. in descending order
                cut = eff_rate * len(scores_sorted)
                thresh = scores_sorted[int(cut)]
        
                # Find fraction of events in the SR which survive a cut on the above threshold (our custom metric!)
                scores_sorted = np.sort(scores_val[Y_val == 1])
                sig_eff_val = 1.0 - 1.0*np.searchsorted(scores_sorted, thresh) / len(scores_sorted)
        
                # Save the signal efficiency
                best_val[k][l].append(sig_eff_val)
    
                # Define filename to save the trained model with the best validation performance
                if (l < k):
                    best_model_filename = "CWoLa_signal_m500_models/models_B%d_%d/best_[" % (counter, run) + str(k) + "," + str(l) + "]_" + str(i) + ".h5"
                else:
                    best_model_filename = "CWoLa_signal_m500_models/models_B%d_%d/best_[" % (counter, run) + str(k) + "," + str(l+1) + "]_" + str(i) + ".h5"
            
                # If this model is the best so far, save it
                if (len(best_val[k][l]) == 0):
                    model.save(best_model_filename)
                elif (sig_eff_val >= np.array(best_val[k][l]).max()):
                    model.save(best_model_filename)

                # Clear memory
                K.clear_session()
                gc.collect()
                del model
            
                pass  # End ntries loop
        
            pass  # End l loop
        
    
        # Pick the best model from the ntries for each validation kfold, and save them together in a list
        best_val_models = []
        best_val_models_hist = []
        
        for l in range(kfolds):
            if (l == k):
                continue
            if (l > k):
                l = l-1
            
            # Define best model from the n = ntries that we trained
            i = np.argmax(best_val[k][l])
            
            # Load the model
            if (l < k):
                model = load_model("CWoLa_signal_m500_models/models_B%d_%d/best_[" % (counter, run) + str(k) + "," + str(l) + "]_" + str(i) + ".h5")
            else:
                model = load_model("CWoLa_signal_m500_models/models_B%d_%d/best_[" % (counter, run) + str(k) + "," + str(l+1) + "]_" + str(i) + ".h5")
            
            # Rename the model
            model.name = "model" + str(k) + str(l) + str(i)
            
            # Save it in the list of models with best validation performance
            best_val_models.append(model)
            best_val_models_hist.append(models_history[k][l][i])
    
            # Plot training history
            plt.plot(best_val_models_hist[l].history['loss'], label='train')
            plt.plot(best_val_models_hist[l].history['val_loss'], label='validation')
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.title('k = %i, l = %i' % (k, l))
            plt.legend(loc='upper right')
            plt.savefig("CWoLa_signal_m500_models/models_B%d_%d/loss_" % (counter, run) + str(k) + "_" + str(l) + ".png")
            plt.show()
    
        # Build up ensemble model
        input_dim = best_val_models[0].layers[0].input.get_shape().as_list()[1]
        singleInput = Input((input_dim,))
        outs = [model_i(singleInput) for model_i in best_val_models]
        outs_merge = Average()(outs)
        merged_model = Model(singleInput, outs_merge)
    
        # Save ensemble models
        ensemble_filename = "CWoLa_signal_m500_models/ensemble_models_B%d_%d/ensemble_" % (counter, run) + str(k) + ".h5"
        merged_model.save(ensemble_filename)
    
        # Save preprocessing information for test data
        X_test_pre = np.array([X_test_pre_mean, X_test_pre_std])
    
        # Save test data
        X_test_list.append(X_test)
        X_test_raw_list.append(X_test_raw)
        
        # Clear memory
        K.clear_session()
        gc.collect()
        del model
        
        pass  # End k loop
    
    
    #######################################################################################################
    ##########################################   Save to file   ###########################################
    #######################################################################################################    
    
    # Create dataset and classes to store important information
    extra = h5py.File('CWoLa_signal_m500_models/extra_info_B%d_%d/extra_info.hdf5' % (counter, run), 'w')

    best_val_tofile = extra.create_dataset('best_val', (5, 4, ntries), dtype='f8')

    X_test_list_tofile = extra.create_dataset('X_test_list', (5, len(X_test_list[0]), 14), dtype='f8')
    X_test_raw_list_tofile = extra.create_dataset('X_test_raw_list', (5, len(X_test_raw_list[0]), 14), dtype='f8')
    signal_extra_raw_tofile = extra.create_dataset('signal_extra_raw', (len(signal_extra), 14), dtype='f8')
    
    bins_tofile = extra.create_dataset('bins', (1, nbin+1), dtype='f8')
    X_test_pre_tofile = extra.create_dataset('X_test_pre', (1, 2, 12), dtype='f8')
    bench_info_tofile = extra.create_dataset('bench_info', (4, ), dtype='f8')
    other_tofile = extra.create_dataset('other', (1, 6), dtype='f8')

    # Fill dataset
    for k in range(kfolds):
        best_val_tofile[k] = best_val[k]
        X_test_list_tofile[k] = X_test_list[k]
        X_test_raw_list_tofile[k] = X_test_raw_list[k]
    
    signal_extra_raw_tofile[:] = signal_extra
    
    bins_tofile[0] = bins
    X_test_pre_tofile[0] = X_test_pre
    bench_info_tofile[:] = np.array([n_signal, n_background, S_B, S_sqrt_B])
    other_tofile[0] = np.array([kfolds, nbin, SR_lower_edge, SR_upper_edge, SB_lower_edge, SB_upper_edge])
    
    extra.close()
    
    #######################################################################################################
    #########################################   Load from file   ##########################################
    #######################################################################################################    
    
    # Load extra info from file
    extra_filename = 'CWoLa_signal_m500_models/extra_info_B%d_%d/extra_info.hdf5' % (counter, run)
    h5_results = h5py.File(extra_filename, 'r')

    # Store extra info
    best_val = h5_results['best_val'][:]
    X_test_list = h5_results['X_test_list'][:]
    X_test_raw_list = h5_results['X_test_raw_list'][:]
    signal_extra_raw = h5_results['signal_extra_raw'][:]
    bins = h5_results['bins'][0,:]
    X_test_pre = h5_results['X_test_pre'][0,:]
    n_signal = int(h5_results['bench_info'][0])
    n_background = int(h5_results['bench_info'][1])
    S_B = h5_results['bench_info'][2]
    S_sqrt_B = h5_results['bench_info'][3]
    kfolds = int(h5_results['other'][0,0])
    nbin = int(h5_results['other'][0,1])
    SR_lower_edge = int(h5_results['other'][0,2])
    SR_upper_edge = int(h5_results['other'][0,3])
    SB_lower_edge = int(h5_results['other'][0,4])
    SB_upper_edge = int(h5_results['other'][0,5])
    
    h5_results.close()

    
    #######################################################################################################
    #############################################   Testing   #############################################
    #######################################################################################################      
    
    # Preprocess extra signal data
    X_test_pre_mean = X_test_pre[0]
    X_test_pre_std = X_test_pre[1]
    signal_extra[:,:12] = (signal_extra_raw[:,:12] - X_test_pre_mean) / X_test_pre_std
    
    # Put extra signal events in kfold list
    sig_size = int(n_signal_extra/kfolds)
    signal_extra_list = []
    signal_extra_list_raw = []
    
    for q in range(kfolds):
        signal_extra_list.append(signal_extra[q*sig_size:(q+1)*sig_size])
        signal_extra_list_raw.append(signal_extra_raw[q*sig_size:(q+1)*sig_size])
        pass
    
    # Test data
    X_ROC = []
    Y_ROC = []
    all_scores = []

    for k in range(kfolds):  # Loop over test
    
        # Load ensemble model
        ensemble_model = load_model("CWoLa_signal_m500_models/ensemble_models_B%d_%d/ensemble_" % (counter, run) + str(k) + ".h5")
    
        # Load test k-fold
        X_test = X_test_list[k]
        X_test_raw = X_test_raw_list[k]
        Y_test = X_test[:,13]
        
        # Load extra signal test k-fold
        X_sig_test = signal_extra_list[k]
        X_sig_test_raw = signal_extra_list_raw[k]
        Y_sig_test = X_sig_test[:,13]
        
        # Combine both
        X_test = np.concatenate((X_test, X_sig_test), axis=0)
        X_test_raw = np.concatenate((X_test_raw, X_sig_test_raw), axis=0)
        Y_test = np.concatenate((Y_test, Y_sig_test), axis=0)
        
        X_test, X_test_raw, Y_test = shuffle_3D(X_test, X_test_raw, Y_test)
    
        # Save data and truth labels for ROC curve
        X_ROC.append(X_test)
        Y_ROC.append(Y_test)
    
        # Make predictions with ensemble model on test kfold
        print("Testing %i..." % k)
        scores_test = ensemble_model.predict(X_test[:,:12], batch_size = nbatch).flatten()
    
        # Save scores
        all_scores.append(scores_test)

    # Transform some lists to arrays
    X_ROC = np.array(X_ROC).reshape(len(X_ROC)*len(X_ROC[0]), 14)
    Y_ROC = np.array(Y_ROC).reshape(len(Y_ROC)*len(Y_ROC[0]), 1)
    all_scores = np.array(all_scores).reshape(len(all_scores)*len(all_scores[0]))

    # Calculate ROC curve
    ROC_scores = all_scores
    fpr, tpr, thresholds = roc_curve(Y_ROC, ROC_scores)
    
    # Save auc
    auc_number = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.plot(tpr, 1-fpr, label="CWoLa (AUC = %.2f)" % auc_number)
    plt.legend(loc='lower left')
    plt.title('S/B = %.4f' % S_B)
    plt.savefig("CWoLa_signal_m500_models/models_B%d_%d/ROC_curve" % (counter, run) + ".png")
    plt.show()
    
    # Plot scores for signal and background separately
    scores_mean = np.mean(all_scores)
    n1 = len(all_scores[all_scores < scores_mean])/len(all_scores)
    n2 = len(all_scores[all_scores > scores_mean])/len(all_scores)
    plt.hist(all_scores, bins=60, range=(0.4, 0.6), alpha=0.5, color='r', label='N < %.3f = %.2f \nN > %.3f = %.2f \nsig_eff_train = %.3f \nsig_eff_val = %.3f ' % (scores_mean, n1, scores_mean, n2, sig_eff_train_mean, sig_eff_val_mean))
    plt.xlabel(r'scores')
    plt.ylabel('Events')
    plt.legend(loc='upper right')
    plt.title('Score distribution')
    plt.savefig("CWoLa_signal_m500_models/models_B%d_%d/score_distribution" % (counter, run) + ".png")
    plt.show()

    K.clear_session()
    gc.collect()
    del(ensemble_model)
    
    #######################################################################################################
    ##########################################   Save to file   ###########################################
    #######################################################################################################    
    
    # Create dataset and classes to store important information
    extra = h5py.File('CWoLa_signal_m500_models/extra_info_B%d_%d/test_results.hdf5' % (counter, run), 'w')
    
    info_tofile = extra.create_dataset('info', (1, ), dtype='f8')
    info_tofile[:] = auc_number
    
    extra.close()
    
    counter += 1